In [ ]:
import os
import re
from tqdm import tqdm

from bdv_toolz.cli import nd2_to_bdv
import sys

from bdv_toolz.bdv_creation import create_bdv_n5_multi_tile


In [ ]:
import xxhash
import os
import time

def files_match(src_path, dest_path):
    """Compare two files by size first, then by xxhash if needed."""
    
    if os.path.getsize(src_path) != os.path.getsize(dest_path):
        return False  # Files are definitely different
    
    # Use xxhash for comparison
    return compute_xxhash(src_path) == compute_xxhash(dest_path)

def compute_xxhash(file_path, chunk_size=65536):
    """Compute xxhash checksum for a given file with progress tracking."""
    hasher = xxhash.xxh64()
    
    # Get file size and calculate total number of chunks
    file_size = os.path.getsize(file_path)
    total_chunks = (file_size + chunk_size - 1) // chunk_size  # This is equivalent to math.ceil(file_size / chunk_size)
    
    print(f"Total number of chunks to process: {total_chunks}")
    
    total_processed_chunks = 0
    
    try:
        with open(file_path, "rb") as f:
            while chunk := f.read(chunk_size):
                hasher.update(chunk)
                total_processed_chunks += 1
                
                # Print progress every 1000 chunks
                if total_processed_chunks % 1000 == 0:
                    print(f"Chunks processed: {total_processed_chunks}/{total_chunks}")
        
        return hasher.hexdigest()
    
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None


In [ ]:

# Define base paths
base_src_folder_path = "/run/user/1001/gvfs/smb-share:server=scratch4.ista.ac.at,share=scratch/siegegrp/_ImageDrop/NIKON-W1-03/Thomas/nd_seq"
base_dest_folder_path = "/mnt/gdrive/ThNe/_Bioimaging/development_retina"

# Define file names
src_file = "slide_P10_M_1001.nd2"
dest_file = "slide_P10_M_1001.nd2"

# Construct full file paths
src_path = os.path.join(base_src_folder_path, src_file)
dest_path = os.path.join(base_dest_folder_path, dest_file)

# Check if files match
if files_match(src_path, dest_path):
    print("✅ Files match!")
else:
    print("❌ Files are different!")
